**1. Визначити сутність і виконати аналіз функцій парного блокуючого обміну.**

Парний блокуючий обмін - це базова модель взаємодії двох процесів у MPI, коли один процес надсилає повідомлення, а інший його приймає, і обидві операції є блокуючими, тобто процес зупиняється, доки виклик не завершиться.

У парному обміні беруть участь два процеси: відправник і одержувач.

Відправник викликає функцію MPI_Send, передаючи адресу буфера, тип даних, кількість елементів, тег повідомлення та ранг процесу-одержувача.

Одержувач викликає MPI_Recv з аналогічними параметрами, вказуючи, від кого він чекає повідомлення та який тип і тег має відповідати.

Блокуюча відправка означає, що процес-відправник не продовжить виконання, поки MPI не буде повністю впевнена, що параметри буфера можна безпечно змінювати - або тому, що дані вже гарантовано передані, або тому, що вони скопійовані у внутрішній буфер MPI.

Важливою частиною парного обміну є узгодженість параметрів: обидві сторони мають співпасти за типом даних, розміром та тегом. Це забезпечує коректність і запобігає неоднозначності.

Блокуючі виклики забезпечують ряд важливих властивостей.
 * Кожен MPI_Recv отримає саме те повідомлення, якого він очікує.
 * Вони забезпечують локальну синхронізацію між процесами. Хоча MPI_Send не зобов’язаний чекати фактичного прийому, у більшості реалізацій прості повідомлення передаються одразу, і відправник блокується тільки до моменту копіювання в буфер MPI.
 * Парний блокуючий обмін може бути джерелом взаємних блокувань (deadlock), якщо обидва процеси викликають MPI_Send, не маючи активних прийомів. Наприклад, двостороння відправка без попереднього MPI_Recv може заблокувати програму, якщо реалізація MPI не має достатньо буфера для обох повідомлень.
 * Також, важливо розуміти його семантику: завершення відправки не означає, що повідомлення доставлене одержувачу. Завершення прийому означає, що дані гарантовано знаходяться у буфері одержувача.

In [1]:
!pip install mpi4py


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


**Тестова програма для перевірки роботи бібліотеки в середовищі Jupiter Notebook**

In [2]:
%%writefile test_mpi.py
from mpi4py import MPI
comm = MPI.COMM_WORLD
print(f"Rank: {comm.Get_rank()} | Size: {comm.Get_size()}")

Overwriting test_mpi.py


In [3]:
!mpiexec -n 4 python test_mpi.py

Rank: 1 | Size: 4
Rank: 0 | Size: 4


Rank: 2 | Size: 4
Rank: 3 | Size: 4




**2. Побудувати програму-шаблон для парного блокуючого обміну**

In [4]:
%%writefile task2.py
from mpi4py import MPI

# function to return whether a number of a process is odd or even
def odd(number):
    if (number % 2) == 0:
        return False
    else :
        return True

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code"
    numProcesses = comm.Get_size()  #total number of processes running"
    myHostName = MPI.Get_processor_name()  #machine name running the code"

    # num of processes must be even
    if numProcesses > 1 and not odd(numProcesses):
        sendValue = id

         #odd processes receive from their paired 'neighbor', then send
        if odd(id):
            comm.send(sendValue, dest=id - 1)
            receivedValue = comm.recv(source=id - 1)

        #even processes receive from their paired 'neighbor', then send
        else:
            receivedValue = comm.recv(source=id + 1)
            comm.send(sendValue, dest=id + 1)

        print(
            "Process {} of {} on {} computed {} and received {}".format(
                id, numProcesses, myHostName, sendValue, receivedValue
            )
        )

    else:
        if id == 0:
            print("Please run this program with a positive even number of processes.")

if __name__ == "__main__":
    main()


Overwriting task2.py


In [5]:
!mpiexec -n 6 python task2.py

Process 2 of 6 on DESKTOP-L6FRMTM computed 2 and received 3
Process 4 of 6 on DESKTOP-L6FRMTM computed 4 and received 5

Process 3 of 6 on DESKTOP-L6FRMTM computed 3 and received 2

Process 5 of 6 on DESKTOP-L6FRMTM computed 5 and received 4


Process 0 of 6 on DESKTOP-L6FRMTM computed 0 and received 1
Process 1 of 6 on DESKTOP-L6FRMTM computed 1 and received 0


Як можемо бачити з цього прикладу, дані попарно відправляються між процесами один одному.

Процес 1, відправив процесу 0 значення свого рангу "1", а процес 0 в свою чергу, відправив процесу 1 аналогічно - "0"

**Атрибути функцій обміну (Send / Recv):**

У блокуючому парному обміні беруть участь такі параметри:

* buf - область памʼяті з даними для передачі або прийому.
* count - кількість елементів у буфері.
* datatype - MPI-тип одного елемента (MPI_INT, MPI_DOUBLE, масиви NumPy і т.д.).
* dest / source - ранг процесу, якому надсилаємо або від якого очікуємо повідомлення.
* tag - ціле число для логічної ідентифікації виду повідомлення.
* comm - комунікатор (наприклад, MPI_COMM_WORLD).
* status (тільки для Recv) - структура, у якій після завершення зберігається фактичний відправник, тег та код завершення.

**Аналіз типів даних:**
1. MPI не робить автоматичного перетворення типів: тип відправника і одержувача має збігатися.
2. count і datatype однозначно визначають обсяг памʼяті, що MPI повинна передати.
3. Якщо count = 0, операція дозволена й завершується миттєво.
4. Розмір приймального буфера повинен бути не меншим, ніж фактичний розмір отриманих даних - інакше помилка.
5. У mpi4py:
  * send(obj) - обʼєкт серіалізується, тип визначається автоматично.
  * Send([...], MPI.INT) - передаються сирі дані з чітким типом і розміром.
6. Типи даних визначають спосіб розбору байтів, тому невідповідність типів призводить до некоректного читання або помилки.

**Семантичний аналіз парного блокуючого обміну**

1. Операція MPI_Send блокується, поки MPI не гарантує, що буфер можна змінювати (дані або передані, або скопійовані в буфер MPI).
2. MPI_Recv блокується, поки дані не будуть повністю отримані і записані в приймальний буфер.
3. Відправник та одержувач повинні бути логічно узгоджені: однаковий tag, однаковий datatype, однаковий count, коректно вказані source/dest
4. Порушення симетрії веде до дедлоку (наприклад, обидва процеси одночасно викликають Send і жоден не виконує Recv).
5. Завершення Send не означає, що одержувач вже отримав повідомлення - лише те, що відправник може використовувати буфер.
6. Завершення Recv означає, що дані гарантовано у приймальному буфері.
7. Парний блокуючий обмін створює локальну синхронізацію між процесами: один процес не може рухатись далі, доки пара не виконає узгоджену операцію.

**Змінимо в програмі порядок виконання функцій для парних рангів, щоб продемострувати дедлок**

In [6]:
%%writefile task2_deadlock.py
from mpi4py import MPI

# function to return whether a number of a process is odd or even
def odd(number):
    if (number % 2) == 0:
        return False
    else :
        return True

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code"
    numProcesses = comm.Get_size()  #total number of processes running"
    myHostName = MPI.Get_processor_name()  #machine name running the code"

    # num of processes must be even
    if numProcesses > 1 and not odd(numProcesses):
        sendValue = id

         #odd processes receive from their paired 'neighbor', then send
        if odd(id):
            receivedValue = comm.recv(source=id - 1)
            comm.send(sendValue, dest=id - 1)

        #even processes receive from their paired 'neighbor', then send
        else:
            receivedValue = comm.recv(source=id + 1)
            comm.send(sendValue, dest=id + 1)

        print(
            "Process {} of {} on {} computed {} and received {}".format(
                id, numProcesses, myHostName, sendValue, receivedValue
            )
        )

    else:
        if id == 0:
            print("Please run this program with a positive even number of processes.")

if __name__ == "__main__":
    main()


Overwriting task2_deadlock.py


In [7]:
!mpiexec -n 6 python task2_deadlock.py

^C


Як бачимо процеси зависли - дедлок

**3. Виконати аналіз інших комунікаційних режимів. Показати зміни в шаблоні для їх реалізації.**

**У MPI існує чотири режими відправки повідомлень. Головна відмінність між ними — це умова завершення функції відправки (коли функція повертає управління програмі)**

* Синхронний (Synchronous), ssend - Рукостискання. Функція завершується тільки тоді, коли процес-одержувач розпочав прийом повідомлення (викликав recv). Це гарантує, що обидва процеси досягли точки обміну. Найбезпечніший, але може бути повільнішим через очікування
* Буферизований (Buffered), bsend - Через буфер користувача. Повідомлення копіюється у спеціально виділений  буфер, і функція повертається миттєво. Не треба залежати від того, чи готовий одержувач. Ризик: якщо буфер переповниться, виникне помилка.
* По готовності (Ready), rsend - Оптимістичний. Можна викликати лише якщо ви впевнені на 100%, що одержувач вже викликав recv. Якщо ні — поведінка не визначена (помилка або краш). Використовується вкрай рідко для оптимізації.

In [18]:
%%writefile task3.py
from mpi4py import MPI
import time
import sys

def odd(number):
    if (number % 2) == 0:
        return False
    else :
        return True

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()
    numProcesses = comm.Get_size()
    myHostName = MPI.Get_processor_name()

    if numProcesses < 2 or odd(numProcesses):
        if id == 0:
            print("Error: Please run with an even number of processes.")
        sys.exit(0)

    # Дані для відправки
    sendValue = id
    neighbor = id - 1 if odd(id) else id + 1

    # ==========================================
    # 1. СИНХРОННИЙ РЕЖИМ (Synchronous - ssend)
    # ==========================================
    if id == 0:
        print("\n--- MODE 1: SYNCHRONOUS (SSEND) ---")
        print("Waiting for handshake...")
    comm.Barrier() # Синхронізація перед початком етапу

    # Логіка як у вашому шаблоні: Непарні шлють, Парні приймають
    if odd(id):
        # ssend заблокується, поки neighbor не викличе recv
        comm.ssend(sendValue, dest=neighbor)
        receivedValue = comm.recv(source=neighbor)
    else:
        receivedValue = comm.recv(source=neighbor)
        comm.ssend(sendValue, dest=neighbor)

    print(f"[Sync] P{id} finished exchange.")
    comm.Barrier() # Чекаємо завершення етапу всіма процесами

    # ==========================================
    # 2. БУФЕРИЗОВАНИЙ РЕЖИМ (Buffered - bsend)
    # ==========================================
    if id == 0:
        print("\n--- MODE 2: BUFFERED (BSEND) ---")
        print("Allocating buffers...")

    # 1. Виділення та приєднання буфера
    # Розмір: дані + службова інформація MPI
    buf_size = 1024 + MPI.BSEND_OVERHEAD
    buf = bytearray(buf_size)
    MPI.Attach_buffer(buf)

    comm.Barrier()

    if odd(id):
        # bsend копіює в buf і повертається миттєво
        comm.bsend(sendValue, dest=neighbor)
        receivedValue = comm.recv(source=neighbor)
    else:
        receivedValue = comm.recv(source=neighbor)
        comm.bsend(sendValue, dest=neighbor)

    # 2. Від'єднання буфера (гарантує, що дані пішли)
    MPI.Detach_buffer()
    print(f"[Buffered] P{id} finished exchange.")
    comm.Barrier()

    # ==========================================
    # 3. РЕЖИМ ПО ГОТОВНОСТІ (Ready - rsend)
    # ==========================================
    if id == 0:
        print("\n--- MODE 3: READY (RSEND) ---")
        print("Ensuring receiver is ready before sending...")

    comm.Barrier()

    # Увага: Для rsend отримувач ОБОВ'ЯЗКОВО має вже чекати (recv).
    # Ми використовуємо time.sleep(), щоб гарантувати порядок.

    if odd(id):
        # Крок 1: Непарний відправляє
        # Чекаємо, щоб Парний точно встиг викликати recv
        time.sleep(2.0) # Було 0.2, ставимо 2.0 для надійності
        comm.rsend(sendValue, dest=neighbor)

        # Крок 2: Непарний приймає
        # Одразу стаємо на прийом, щоб Парний міг відправити нам
        receivedValue = comm.recv(source=neighbor)
    else:
        # Одразу викликаємо recv -> стаємо "Ready"
        receivedValue = comm.recv(source=neighbor)

        # Крок 2: Парний відправляє
        # Чекаємо, щоб Непарний встиг перейти до recv
        time.sleep(2.0) # Було 0.2
        comm.rsend(sendValue, dest=neighbor)


    print(f"[Ready] P{id} finished exchange.")
    comm.Barrier()

    # Фінальний вивід (як у шаблоні)
    if id == 0: print("\n--- RESULTS ---")
    comm.Barrier()
    time.sleep(0.1) # Щоб вивід не перемішався
    print(
        "Process {} of {} on {} | Final Recv: {}".format(
            id, numProcesses, myHostName, receivedValue
        )
    )

if __name__ == "__main__":
    main()

Overwriting task3.py


In [19]:
!mpiexec -n 6 python task3.py

^C
